In [2]:
# https://keras.io/
!pip install -q keras
import keras

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls /content/drive/

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
Files in Drive:
Colab Notebooks  created.txt  data  Louis共享文件夹  M.Sc-DIRO-UdeM  test.ods


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#%%
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import gzip


batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

import os
os.chdir("/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1")
# os.chdir("/Users/louis/Google Drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1")
# os.chdir("/Users/fanxiao/Google Drive/UdeM/IFT6135 Representation Learning/homework1/programming part ")
print(os.getcwd())



/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1


In [23]:

#%%
# Vectorize the data.

input_characters = set()
target_characters = set()
def loadData(corpuspath,size=None):
    input_texts = []
    target_texts = []
    with gzip.open(corpuspath, 'rt', encoding='ISO-8859-1') as f:
        lines = f.read().split('\n')
    input_phrase=[]
    target_phrase=[]
    # for line in lines[: min(num_samples, len(lines) - 1)]:
    i=0
    for line in lines:
        if not line.startswith('#begin') and not line.startswith('#end') and len(line.split('\t'))>1:
            line=line.encode("ascii", errors="ignore").decode()
            target_word, input_word = line.split('\t')
            input_word=input_word.strip().lower()
            target_word=target_word.strip().lower()
            input_phrase.append(input_word)
            target_phrase.append(target_word)
            input_phrase.append(' ')
            target_phrase.append(' ')
            if input_word=='.':
                # We use "tab" as the "start sequence" character
                # for the targets, and "\n" as "end sequence" character.
                input_texts.append(''.join(input_phrase))
                target_texts.append('\t'+''.join(target_phrase) + '\n')
                input_phrase=[]
                target_phrase=[]
                i+=1

            for char in input_word:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_word:
                if char not in target_characters:
                    target_characters.add(char)
            if size is not None and i>size: break
            
    return input_texts,target_texts
    
input_texts,target_texts=loadData('data/train-1544.gz',6000)
size_train=len(input_texts)
test_input_texts,test_target_texts=loadData('data/test-2834.gz',500)
size_test=len(test_input_texts)


input_texts=np.array(input_texts)
target_texts=np.array(target_texts)
# np.random.seed(456)
# indexs = np.arange(0,len(input_texts))
# np.random.shuffle(indexs)
# input_texts = input_texts[indexs[0:size_train]]
# target_texts=target_texts[indexs[0:size_train]]

test_input_texts=np.array(test_input_texts)
test_target_texts=np.array(test_target_texts)

#input_texts,target_texts: list of phrases
#input_token_index: key-values of char
#encoder_input_data:3dimension, d1=phrase,d2=char index in phrase,d3=index of list of candidate chars
# print(input_texts[0:2])
# print(target_texts[0:2])

input_characters.add(' ')
target_characters.add(' ')
target_characters.add('\t')
target_characters.add('\n')
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of sentence samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print('sentence of training data:', size_train)
print('sentence of test data:', size_test)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


Number of sentence samples: 6001
Number of unique input tokens: 60
Number of unique output tokens: 62
Max sequence length for inputs: 817
Max sequence length for outputs: 841
sentence of training data: 6001
sentence of test data: 501


In [0]:

# print(input_token_index)
# print(target_token_index)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [7]:

#%%
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
# model.save('s2s.h5')


Train on 4800 samples, validate on 1201 samples
Epoch 1/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.4203 - val_loss: 0.4775
Epoch 2/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.3805 - val_loss: 0.4321
Epoch 3/100
4800/4800 [==============================] - 204s 43ms/step - loss: 0.3438 - val_loss: 0.3942
Epoch 4/100
4800/4800 [==============================] - 204s 42ms/step - loss: 0.3214 - val_loss: 0.3779
Epoch 5/100
1408/4800 [=======>......................] - ETA: 2:12 - loss: 0.3116

4800/4800 [==============================] - 203s 42ms/step - loss: 0.3073 - val_loss: 0.3663
Epoch 6/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.2967 - val_loss: 0.3573
Epoch 7/100
4800/4800 [==============================] - 201s 42ms/step - loss: 0.2875 - val_loss: 0.3536
Epoch 8/100
4800/4800 [==============================] - 201s 42ms/step - loss: 0.2790 - val_loss: 0.3483
Epoch 9/100
4096/4800 [========================>.....] - ETA: 27s - loss: 0.2718

4800/4800 [==============================] - 203s 42ms/step - loss: 0.2714 - val_loss: 0.3415
Epoch 10/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.2646 - val_loss: 0.3367
Epoch 11/100
4800/4800 [==============================] - 199s 42ms/step - loss: 0.2583 - val_loss: 0.3312
Epoch 12/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.2527 - val_loss: 0.3283
Epoch 13/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.2474

4800/4800 [==============================] - 204s 42ms/step - loss: 0.2476 - val_loss: 0.3227
Epoch 14/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.2428 - val_loss: 0.3221
Epoch 15/100
4800/4800 [==============================] - 204s 42ms/step - loss: 0.2387 - val_loss: 0.3179
Epoch 16/100
4800/4800 [==============================] - 204s 43ms/step - loss: 0.2345 - val_loss: 0.3143
Epoch 17/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.2311

4800/4800 [==============================] - 204s 42ms/step - loss: 0.2308 - val_loss: 0.3140
Epoch 18/100
4800/4800 [==============================] - 205s 43ms/step - loss: 0.2274 - val_loss: 0.3091
Epoch 19/100
4800/4800 [==============================] - 204s 42ms/step - loss: 0.2242 - val_loss: 0.3127
Epoch 20/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.2217 - val_loss: 0.3059
Epoch 21/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.2176

...

4800/4800 [==============================] - 204s 43ms/step - loss: 0.1464 - val_loss: 0.2909
Epoch 82/100
4800/4800 [==============================] - 204s 42ms/step - loss: 0.1455 - val_loss: 0.2905
Epoch 83/100
4800/4800 [==============================] - 204s 43ms/step - loss: 0.1449 - val_loss: 0.2889
Epoch 84/100
4800/4800 [==============================] - 206s 43ms/step - loss: 0.1439 - val_loss: 0.2883
Epoch 85/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.1436

4800/4800 [==============================] - 206s 43ms/step - loss: 0.1431 - val_loss: 0.2901
Epoch 86/100
4800/4800 [==============================] - 204s 43ms/step - loss: 0.1424 - val_loss: 0.2903
Epoch 87/100
4800/4800 [==============================] - 205s 43ms/step - loss: 0.1416 - val_loss: 0.2913
Epoch 88/100
4800/4800 [==============================] - 205s 43ms/step - loss: 0.1410 - val_loss: 0.2927
Epoch 89/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.1390

4800/4800 [==============================] - 204s 42ms/step - loss: 0.1401 - val_loss: 0.2918
Epoch 90/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.1395 - val_loss: 0.2955
Epoch 91/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.1388 - val_loss: 0.2940
Epoch 92/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.1382 - val_loss: 0.2992
Epoch 93/100
4416/4800 [==========================>...] - ETA: 15s - loss: 0.1377

4800/4800 [==============================] - 203s 42ms/step - loss: 0.1376 - val_loss: 0.2982
Epoch 94/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.1368 - val_loss: 0.2982
Epoch 95/100
4800/4800 [==============================] - 201s 42ms/step - loss: 0.1362 - val_loss: 0.2983
Epoch 96/100
4800/4800 [==============================] - 201s 42ms/step - loss: 0.1356 - val_loss: 0.2990
Epoch 97/100
4416/4800 [==========================>...] - ETA: 14s - loss: 0.1349

4800/4800 [==============================] - 201s 42ms/step - loss: 0.1350 - val_loss: 0.2977
Epoch 98/100
4800/4800 [==============================] - 203s 42ms/step - loss: 0.1344 - val_loss: 0.3005
Epoch 99/100
4800/4800 [==============================] - 204s 42ms/step - loss: 0.1338 - val_loss: 0.2987
Epoch 100/100
4800/4800 [==============================] - 202s 42ms/step - loss: 0.1332 - val_loss: 0.3016


In [5]:
print(os.getcwd())
from keras.models import load_model
model.save('output-lsmtm/model1-lstm-4800samples-100epochs.h5')
# del model  # deletes the existing model

/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1


/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2364: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [0]:

#%%
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence
  
#%%
'''
Functions of Evalutate the prediction
'''
def count_accuracy(pred_sents,target_sents):
    count_accu=0
    total=0
    for pred_sent,target_sent in zip(pred_sents,target_sents):
        pred_list=pred_sent.split(' ')
        targ_list=target_sent[2:].split(' ')
        for pred_token,target_token in zip(pred_list,targ_list):
            total+=1
            if pred_token==target_token:
                count_accu+=1
    return count_accu, total

In [9]:

#%%
print('-------predict train data:')
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-------predict train data:
-
Input sentence: the rimatara reed warbler ( acrocephalus rimitara ) be a species of old world warbler in the acrocephalidae family . 
Decoded sentence: tht iming the most of game of the english are exception to parriage that the company state , which is a community foundation . 

-
Input sentence:  it be find only in french polynesium . 
Decoded sentence:  it is found only in malaysia . 

-
Input sentence:  its natural habitat be subtropical or tropical dry forest and swamp . 
Decoded sentence:  its natural habitat is subtropical or tropical dry shrubland . 

-
Input sentence:  the rodrigues warbler ( acrocephalus rodericanus ) be a species of old world warbler in the acrocephalidae family . 
Decoded sentence:  the red sox went of the season , the season , he was also a marries compositions . 

-
Input sentence:  it be find only on a small island call rodrigues , which belong to mauritius and use to be plentiful there . 
Decoded sentence:  it is found only 

In [20]:
# print(os.getcwd())
# from keras.models import load_model
# model=load_model('output-lstm/model1-lstm-4016samples-100epochs.h5')

/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1


In [0]:
#%%
# print('-------predict train data:')
# train_pred_sents=[]
# for seq_index in range(500):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-- No. ',seq_index)
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)
#     train_pred_sents.append(decoded_sentence)

# train_acc_count,count_total=count_accuracy(train_pred_sents,input_texts)
# print('Accuracy on LSTM1 predicteur, train data:', train_acc_count,'/', count_total,'=',train_acc_count/count_total)




In [0]:

test_encoder_input_data = np.zeros(
    (len(test_input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
test_decoder_input_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
test_decoder_target_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


for i, (test_input_text, test_target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(test_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [11]:


print('-----predict test data:')
test_pred_sents=[]
for seq_index in range(200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = test_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-- No. ',seq_index)
    print('Input test sentence:', test_input_texts[seq_index])
    print('Decoded test sentence:', decoded_sentence)
    test_pred_sents.append(decoded_sentence)





-----predict test data:
-- No.  0
Input test sentence: shorty 's lunch be a washington , pennsylvania-based hot dog lunch counter . 
Decoded test sentence:  hs it was discovered in the film rease . 

-- No.  1
Input test sentence:  a `` local landmark , '' shorty 's have be own by the alexas family since the 1930 . 
Decoded test sentence:  a siggle foundation store and south american baseball . 

-- No.  2
Input test sentence:  it have two location , include the main facility on west chestnut street in washington , as well as in canton . 
Decoded test sentence:  it has a settlement in the american league with a record of 72 wins and 70 losses . 

-- No.  3
Input test sentence:  the main restaurant boast old wooden booth , a dining counter , as well a large storefront , show the grill to pedestrian . 
Decoded test sentence:  the main tailer was a senate of emberom , as a structures and service and in survic , song , the state of the community foundations . 

-- No.  4
Input test sentenc

-- No.  26
Input test sentence:  all tell , the project would have be as ambitious as the nearby southpointe development . 
Decoded test sentence:  all of the center of the season , the season , the most of recording and and one of the first temple mount . 

-- No.  27
Input test sentence:  the $ 100 million project would have require the demolition of several building through eminent domain , include the building housing shorty 's original 34 chestnut street location , and would have significant tax break through a tax increment financing program . 
Decoded test sentence:  the new comes from 1967 to 1999 , as the club was founded in 1994 . 

-- No.  28
Input test sentence:  in spite of offer by the developer to assist with move cost , the alexas family refuse to move , gather 23,860 signature in a petition to keep they current location , a amazing number in a city with a population of 16,000 . 
Decoded test sentence:  in the 2008 southern seemates struck , the rest of the centra there

-- No.  51
Input test sentence:  the chapel of st. froiln be in renaissance style , date to the 17th century . 
Decoded test sentence:  the club is located in the town of again . 

-- No.  52
Input test sentence:  notable be the choir , build by francisco de moure ( early 17th century ) . 
Decoded test sentence:  noth are received a provides a new starring from the center of the season . 

-- No.  53
Input test sentence:  surround channel be audio channel in surround sound multichannel audio . 
Decoded test sentence:  surree leaf warbler ( phylloscopus submanis ) is a species of old world warbler in the phylloscopidae family . 

-- No.  54
Input test sentence:  they primarily serve to deliver ambience and diffuse sound in a film or music soundtrack . 
Decoded test sentence:  they released the season , he was also a marries compositions . 

-- No.  55
Input test sentence:  dolby stereo ( 1975 ) be the first standard cinema sound system use a single matrixed mono rear channel ( note disn

-- No.  76
Input test sentence:  jbl state that surround speaker should be space evenly along the side and back wall , start at no closer than 1/3 of the room length from the screen ( so the surround do not interfere with the front channel ) and should be located at about 10 -- 12 foot high and angle down to face the opposite wall-floor boundary . 
Decoded test sentence:  je canso son in extensive district of anter to a hitse family in the manager was a single chart . 

-- No.  77
Input test sentence:  the yamaha xjr1200 be a motorcycle manufacture by the yamaha motor company . 
Decoded test sentence:  the album was one of the first team 's contain and several side . 

-- No.  78
Input test sentence:  in the early 1990s when honda have the honda cb1000 , the xjr be design to compete with the high power naked bike already on the market . 
Decoded test sentence:  in the 2008 southern seems and the cate of the season , and the season , the mang collect of south africa . 

-- No.  79
Input

-- No.  101
Input test sentence:  its administrative center be the rural locality ( a selo ) of pirovskoye . 
Decoded test sentence:  its duret of the entity and a record of 1974 , and the state of the college football league the state of the season . 

-- No.  102
Input test sentence:  population : 9,150 ( 2002 census ) ; the population of pirovskoye account for 39.7 % of the district 's total population . 
Decoded test sentence:  population tuilding are reveloped a currianam composer . 

-- No.  103
Input test sentence:  the district be found on april 4 , 1924 . 
Decoded test sentence:  the district of the red sox fanishity in the album was released in 1979 . 

-- No.  104
Input test sentence:  as of 2013 , the head of the district and the chairman of the district council be alexander i . 
Decoded test sentence:  as of the rease to the state of extending in the 1980s , when him the company state of the community foundations . 

-- No.  105
Input test sentence:  yevseyev . 
Decoded te

-- No.  128
Input test sentence:  he move to california and live on fair oaks avenue in south pasadena , california . 
Decoded test sentence:  he massioned the first construction of the destriction of the community foundations are new york with a providen and received a created of the song . 

-- No.  129
Input test sentence:  he be active in south pasadena civic and political affair . 
Decoded test sentence:  he was also a marrie cross . 

-- No.  130
Input test sentence:  in 1938 , he receive a patent for a ornamental spoon holder . 
Decoded test sentence:  in 1991 , the population of koinise was around 164 inhabitants . 

-- No.  131
Input test sentence:  in june 1940 , speik be find dead by hang in a surgical supply factory in south pasadena . 
Decoded test sentence:  in july 2007 , the center of the state of a structures and society of pale , but also and detacked on the coal boy , and a substantive of the community foundation . 

-- No.  132
Input test sentence:  he be discover h

-- No.  157
Input test sentence:  springfield castle , broadford , be situate in the west part of county limerick , ireland , close to the town of newcastlewest . 
Decoded test sentence:  spiciph was comminted the breeding season , the season , the season , and the second to be found in the caled with a record of 1978 , and the season , he was around 2007 millities and the album and the committee of agries . 

-- No.  158
Input test sentence:  the castle be in fact a group of building around a courtyard , comprise 2 stone keep , or tower ; one 15th century , one 18th century , a range of workshop and stable and the modernised remains of a 18th century servant wing . 
Decoded test sentence:  the club is located in the town of again . 

-- No.  159
Input test sentence:  in 1280 the norman fitzgerald family settle at the site , then know as gort na tiobrad . 
Decoded test sentence:  in 1991 , the population of kormank was around 168 inhabitants . 

-- No.  160
Input test sentence:  the fi

-- No.  183
Input test sentence:  it be replace by the company three month later in what be consider a formidable feat of be re-built in twelve day after construction have start . 
Decoded test sentence:  it is found only in malaysia . 

-- No.  184
Input test sentence:  the current bridge , build from 1928 -- 1930 , be the last of the rail bridge across the bay or kearny point at its northern end . 
Decoded test sentence:  the carres manor care he corpart of the season , and the second to be found in the caled with a record of 1978 , and the season , which are not described and and the community foundation . 

-- No.  185
Input test sentence:  the crrnj newark bay bridge downstream and the newark and new york railroad bridge upstream have be dismantle . 
Decoded test sentence:  the centerr 's career was presented to be the dentation of the season , and the season , and the season , the mang collect of south africa . 

-- No.  186
Input test sentence:  kandidato ( english : candidate )

In [36]:

# input_texts = []
dec_texts = []
with open('output-lstm/prediction200.txt', 'rt', encoding='ISO-8859-1') as f:
  lines = f.read().split('\n')
for line in lines:
#   if line.startswith('--'): continue
#   if line.startswith('Input test sentence:'):
#     input_texts.append(line[21:])
  if line.startswith('Decoded test sentence:'):
    dec_texts.append(line[24:])
i=0
for dec1,dec2 in zip(dec_texts,test_target_texts):
  print (i,'predict: ',dec1)
  print (i,'surface: ',dec2[2:])
  i+=1

0 predict:  hs it was discovered in the film rease . 
0 surface:  horty 's lunch is a washington , pennsylvania-based hot dog lunch counter . 

1 predict:  a siggle foundation store and south american baseball . 
1 surface:  a `` local landmark , '' shorty 's has been owned by the alexas family since the 1930s . 

2 predict:  it has a settlement in the american league with a record of 72 wins and 70 losses . 
2 surface:  it has two locations , including the main facility on west chestnut street in washington , as well as in canton . 

3 predict:  the main tailer was a senate of emberom , as a structures and service and in survic , song , the state of the community foundations . 
3 surface:  the main restaurant boasts old wooden booths , a dining counter , as well a large storefront , showing the grill to pedestrians . 

4 predict:  the temple mount , all of the band , and the second to be population of the season . 
4 surface:  the trademark menu item is an albert 's frank with chili ,

In [35]:


def count_accuracy(pred_sents,target_sents):
    count_accu=0
    total=0
    for pred_sent,target_sent in zip(pred_sents,target_sents):
        pred_list=pred_sent.split(' ')
        targ_list=target_sent[2:].split(' ')
        for pred_token,target_token in zip(pred_list,targ_list):
            total+=1
            if pred_token==target_token:
                count_accu+=1
    return count_accu, total
  
test_acc_count,count_total=count_accuracy(dec_texts,test_target_texts)
print('Accuracy on LSTM1 predicteur, test data:', test_acc_count,'/', count_total,'=',test_acc_count/count_total)


Accuracy on LSTM1 predicteur, test data: 263 / 3400 = 0.07735294117647058
